In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-239222
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-239222


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AT7ZWW5XC to authenticate.


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacity-project"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cluster = ComputeTarget.create(ws, cluster_name, config)

cluster

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-239222', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-239222'), name=udacity-project, id=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourceGroups/aml-quickstarts-239222/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-239222/computes/udacity-project, type=AmlCompute, provisioning_state=Creating, location=southcentralus, tags=None)

In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        'C': uniform(0.2,100.),
        'max_iter': choice(50, 100, 250, 1000)
    }

)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      compute_target=cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4
                    )

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [12]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_46fa8aae-6024-4055-9f1d-3a2a0b6356d0
Web View: https://ml.azure.com/runs/HD_46fa8aae-6024-4055-9f1d-3a2a0b6356d0?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-239222/workspaces/quick-starts-ws-239222&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-07-25T10:32:32.297106][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-25T10:32:32.7960739Z][SCHEDULER][INFO]Scheduling job, id='HD_46fa8aae-6024-4055-9f1d-3a2a0b6356d0_0' 
[2023-07-25T10:32:32.9623292Z][SCHEDULER][INFO]Scheduling job, id='HD_46fa8aae-6024-4055-9f1d-3a2a0b6356d0_1' 
[2023-07-25T10:32:33.0394197Z][SCHEDULER][INFO]Scheduling job, id='HD_46fa8aae-6024-4055-9f1d-3a2a0b6356d0_2' 
[2023-07-25T10:32:33.1283801Z][SCHEDULER][INFO]Scheduling job, id='HD_46fa8aae-6024-4055-9f1d-3a2a0b6356d0_3' 
[2023-07-25T10:32:33.087757][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(f'Best Run id: {best_run.id}')
print(f'Accuracy: {best_run_metrics["accuracy"]}')
print(f'C: {parameter_values[0]}')
print(f'max_iter: {parameter_values[1]}')

best_run.download_file('outputs/hyperdrive_model.pkl', 'hyperdrive_model.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###